# Rough Volatility Neural SDE - Demo Notebook

This notebook demonstrates the key components of the framework:
1. **fBM Generation** - Visualize roughness properties
2. **Neural SDE Training** - Train on toy target
3. **Deep Hedging** - Compare strategies

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import sys
sys.path.insert(0, '..')

from src.noise.fbm import DaviesHarte, generate_fbm_paths
from src.utils.seed import set_all_seeds
from src.agents.baselines import BlackScholesDeltaHedge, NaiveHedgeAgent
from src.metrics.statistics import compute_cvar

set_all_seeds(42)
plt.style.use('seaborn-v0_8-whitegrid')

## 1. Fractional Brownian Motion Visualization

Compare paths for different Hurst parameters to see the roughness effect.

In [ ]:
# Generate fBM paths for different H values
n_steps = 500
T = 1.0
H_values = [0.05, 0.1, 0.2]

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

for ax, H in zip(axes, H_values):
    paths = generate_fbm_paths(n_steps=n_steps, batch_size=5, H=H, T=T)
    time = np.linspace(0, T, n_steps + 1)
    
    for i in range(5):
        ax.plot(time, paths[i], alpha=0.7, linewidth=0.8)
    
    ax.set_title(f'H = {H}', fontsize=14)
    ax.set_xlabel('Time')
    ax.set_ylabel('B^H(t)')
    ax.set_ylim(-2, 2)

plt.suptitle('Fractional Brownian Motion: Roughness vs Hurst Parameter', fontsize=16, y=1.02)
plt.tight_layout()
plt.show()

### Verifying Variance Scaling: Var(B^H_T) = T^{2H}

In [ ]:
# Verify variance scaling
print("Variance Scaling Test")
print("=" * 40)

for H in [0.05, 0.1, 0.25, 0.4]:
    paths = generate_fbm_paths(n_steps=256, batch_size=5000, H=H, T=1.0)
    empirical_var = np.var(paths[:, -1])
    theoretical_var = 1.0 ** (2 * H)  # T^{2H}
    error = abs(empirical_var - theoretical_var) / theoretical_var * 100
    
    print(f"H={H}: Empirical={empirical_var:.4f}, Theory={theoretical_var:.4f}, Error={error:.1f}%")

## 2. Neural SDE Training (Toy Example)

Quick demonstration of training on a toy target distribution.

In [ ]:
from src.models.generator import RoughNeuralSDE
from src.loss.signature import DifferentiablePathLoss

# Create model and loss
model = RoughNeuralSDE(hidden_dim=32, use_euler_heun=True)
loss_fn = DifferentiablePathLoss(include_autocorr=True)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# Generate target paths (rough volatility with H=0.1)
set_all_seeds(42)
fbm = DaviesHarte(n_steps=50, batch_size=256, H=0.1, T=1.0)
target_increments = fbm.sample()
target_paths = np.cumsum(target_increments, axis=1)
target_paths = np.concatenate([np.zeros((256, 1)), target_paths], axis=1)

target_tensor = torch.tensor(target_paths, dtype=torch.float32)

# Training loop (short demo)
print("Training Neural SDE (toy example)...")
losses = []

for epoch in range(20):
    optimizer.zero_grad()
    
    # Generate synthetic paths
    noise = torch.randn(256, 50)
    synthetic = model(noise)
    
    # Compute loss
    loss = loss_fn(synthetic, target_tensor)
    loss.backward()
    optimizer.step()
    
    losses.append(loss.item())
    if epoch % 5 == 0:
        print(f"Epoch {epoch}: Loss = {loss.item():.4f}")

plt.figure(figsize=(8, 4))
plt.plot(losses)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss')
plt.show()

## 3. Hedging Environment Demo

Compare Black-Scholes delta hedge vs. naive hedge.

In [ ]:
from src.hedging.engine import HedgingEnvironment

# Generate price paths
set_all_seeds(42)
n_paths = 500
n_steps = 50
T = 1.0

# Rough volatility paths
fbm = DaviesHarte(n_steps=n_steps, batch_size=n_paths, H=0.1, T=T)
increments = fbm.sample()

paths = np.zeros((n_paths, n_steps + 1))
paths[:, 0] = 1.0
for t in range(n_steps):
    paths[:, t + 1] = paths[:, t] * np.exp(0.2 * increments[:, t])

time_grid = np.linspace(0, T, n_steps + 1)

print(f"Generated {n_paths} rough volatility paths")
print(f"Terminal price range: [{paths[:, -1].min():.2f}, {paths[:, -1].max():.2f}]")

In [ ]:
# Create hedging environment and agents
env = HedgingEnvironment(strike_pct=1.0, transaction_cost=0.001)

bs_agent = BlackScholesDeltaHedge(strike=1.0, sigma=0.2, T=T)
naive_agent = NaiveHedgeAgent(strategy="atm")

# Compute deltas
bs_deltas = bs_agent.compute_deltas(paths, time_grid)
naive_deltas = naive_agent.compute_deltas(paths, time_grid)

# Compute PnL
paths_tensor = torch.tensor(paths, dtype=torch.float32)

bs_pnl = env.compute_pnl(paths_tensor, torch.tensor(bs_deltas, dtype=torch.float32)).numpy()
naive_pnl = env.compute_pnl(paths_tensor, torch.tensor(naive_deltas, dtype=torch.float32)).numpy()

print("\nHedging Results:")
print("=" * 50)
for name, pnl in [("BS Delta", bs_pnl), ("Naive (0.5)", naive_pnl)]:
    cvar, var = compute_cvar(pnl, alpha=0.05)
    print(f"{name:15s}: Mean={np.mean(pnl):.4f}, Std={np.std(pnl):.4f}, CVaR={cvar:.4f}")

In [ ]:
# Plot PnL distributions
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

for ax, (name, pnl, color) in zip(axes, [
    ("BS Delta Hedge", bs_pnl, "steelblue"),
    ("Naive Hedge (δ=0.5)", naive_pnl, "gray")
]):
    ax.hist(pnl, bins=40, color=color, alpha=0.7, edgecolor='black')
    ax.axvline(np.mean(pnl), color='red', linestyle='--', label=f'Mean: {np.mean(pnl):.4f}')
    
    cvar, var = compute_cvar(pnl, alpha=0.05)
    ax.axvline(cvar, color='darkred', linestyle=':', label=f'5% CVaR: {cvar:.4f}')
    
    ax.set_title(name, fontsize=14)
    ax.set_xlabel('P&L')
    ax.set_ylabel('Frequency')
    ax.legend()

plt.suptitle('Hedging P&L Distributions Under Rough Volatility', fontsize=16)
plt.tight_layout()
plt.show()

## Summary

This demo showed:
- **fBM paths** exhibit increasing roughness as H → 0
- **Variance scaling** Var(B^H_T) = T^{2H} verified empirically
- **Neural SDE** trains to match rough path distributions
- **Hedging comparison** shows BS delta outperforms naive strategy

For full experiments, see `experiments/run_*.py` scripts.